In [ ]:
import pandas as pd
from google.colab import files

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import tkinter as tk
from tkinter import filedialog

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Загрузка файла
uploaded = files.upload()

for fn in uploaded.keys():
    print(f'Загружен файл: {fn}')

Saving df_hack_final.csv to df_hack_final.csv
Загружен файл: df_hack_final.csv


In [ ]:
# Чтение
file_name = list(uploaded.keys())[0]  # Получаем загруженный файл

# Загрузка датасета
nikel_data = pd.read_csv(file_name)
# получение первых 5 строк
nikel_data.head()

# Регулярка для разделителей (например, пробелы, табуляции, запятые и т.д.)
# \s+ для пробелов и \t для табуляции
#nikel_data = pd.read_csv(file_name, sep=r'\s+|,|\t', engine='python')  # Указать нужные разделители

# Вывод первых 10 строк
#print("Первые 10 строк данных:")
#print(nikel_data.head(10))

# Сохранение данных в формате CSV
#output_file = 'output.csv'
#ikel_data.to_csv(output_file, index=False)

#print(f'Данные успешно сохранены в {output_file}')

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
0,2024-01-01 00:00:00,2.6097,1.5313,1096.5,1240.597656,692.090942,1.342155,711.999023,1548.71875,0.6232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01 00:15:00,2.5548,1.4842,1123.0,1205.422363,693.616394,1.339809,710.697815,1556.56250,0.6292,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,NaN
2,2024-01-01 00:30:00,2.5109,1.4355,840.0,1188.762573,698.350586,1.339792,707.198547,1548.09375,0.5941,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.970170
3,2024-01-01 00:45:00,2.4765,1.3852,824.0,1151.888672,714.678101,1.342392,707.865540,1538.87500,0.6682,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.968639
4,2024-01-01 01:00:00,2.3585,1.3368,0.0,1104.101318,730.190674,1.337608,700.935059,1545.18750,0.6489,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.974205


### Анализ данных

In [ ]:
# Основные статистические характеристики для каждого числового признака в датасете
nikel_data.describe().round(2)

,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,Ni_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
count,26213.00,26213.00,30336.0,30336.00,30336.00,30336.00,30336.00,30336.00,26083.00,26083.00,...,30335.00,30335.00,30335.00,30335.00,30335.00,30335.0,30335.0,30335.00,30335.00,27759.00
mean,2.71,1.69,1214.0,1346.76,789.38,1.39,811.54,1520.39,0.48,2.84,...,1.63,0.82,0.33,15.65,13.04,3.7,3.5,1.58,0.86,0.95
std,0.28,0.16,304.2,272.75,153.76,0.07,147.98,254.86,0.18,0.20,...,0.33,0.17,0.47,0.90,0.57,0.0,0.0,0.24,0.17,0.02
min,1.65,1.14,0.0,-0.26,-0.17,0.00,-0.08,0.00,0.10,0.68,...,1.00,0.60,0.00,14.00,12.00,3.7,3.5,1.00,0.60,0.59
25%,2.52,1.57,1133.5,1310.45,762.19,1.38,791.39,1523.53,0.40,2.71,...,1.40,0.60,0.00,15.00,13.00,3.7,3.5,1.40,0.70,0.93
50%,2.69,1.69,1274.5,1400.96,817.15,1.40,836.23,1562.81,0.47,2.84,...,1.80,0.80,0.00,15.50,13.00,3.7,3.5,1.60,0.90,0.94
75%,2.89,1.79,1395.5,1475.27,864.99,1.42,875.03,1595.75,0.54,2.97,...,1.80,0.90,1.00,16.50,13.00,3.7,3.5,1.80,1.00,0.97
max,4.05,2.62,2000.0,2089.52,1221.76,1.65,1303.47,2028.38,5.21,3.67,...,2.00,1.20,1.00,17.00,15.00,3.7,3.5,1.80,1.30,1.00


In [ ]:
# Общая информация о датасете
nikel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30336 entries, 0 to 30335
Columns: 130 entries, MEAS_DT to Ni_rec
dtypes: float64(129), object(1)
memory usage: 30.1+ MB


In [ ]:
# Проверка на пропущенные знаки в датасете
nikel_data.isnull().sum()

,0
MEAS_DT,0
Cu_oreth,4123
Ni_oreth,4123
Ore_mass,0
Mass_1,0
...,...
Ni_3.2C_max,1
Ni_3.2C_min,1
Cu_3.2T_max,1
Cu_3.2T_min,1


In [ ]:
# Этот код выполняет кодирование категориальных (строковых) значений в числовые
label_encoders = {}
for column in nikel_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    nikel_data[column] = le.fit_transform(nikel_data[column])
    label_encoders[column] = le

nikel_data.head()

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
0,0,2.6097,1.5313,1096.5,1240.597656,692.090942,1.342155,711.999023,1548.71875,0.6232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.5548,1.4842,1123.0,1205.422363,693.616394,1.339809,710.697815,1556.56250,0.6292,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,NaN
2,2,2.5109,1.4355,840.0,1188.762573,698.350586,1.339792,707.198547,1548.09375,0.5941,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.970170
3,3,2.4765,1.3852,824.0,1151.888672,714.678101,1.342392,707.865540,1538.87500,0.6682,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.968639
4,4,2.3585,1.3368,0.0,1104.101318,730.190674,1.337608,700.935059,1545.18750,0.6489,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.974205


In [ ]:
def fill_missing_values(dataframe):
    """
    Функция для проверки и заполнения пропущенных значений в DataFrame.

    :param dataframe: pd.DataFrame - входной DataFrame
    :return: pd.DataFrame - DataFrame с заполненными пропусками
    """
    for column in dataframe.columns:
        if dataframe[column].isnull().any():  # Проверяем наличие пропусков в столбце
            if dataframe[column].dtype in ['float64', 'int64']:  # Числовые столбцы
                # Заполняем средним значением
                mean_value = dataframe[column].mean()
                dataframe[column].fillna(mean_value, inplace=True)
                print(f"Заполнены пропуски в столбце '{column}' средним значением: {mean_value:.2f}")
            else:  # Категориальные или объектные столбцы
                # Заполняем модой
                mode_value = dataframe[column].mode()[0]
                dataframe[column].fillna(mode_value, inplace=True)
                print(f"Заполнены пропуски в столбце '{column}' модой: {mode_value}")

    return dataframe


# Заполняем пропуски
filled_data = fill_missing_values(nikel_data)

# Выводим результат
print("\nДанные после заполнения пропусков:")
print(filled_data)

<ipython-input-21-7cf36dc0ba58>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe[column].fillna(mean_value, inplace=True)


Заполнены пропуски в столбце 'Cu_oreth' средним значением: 2.71
Заполнены пропуски в столбце 'Ni_oreth' средним значением: 1.69
Заполнены пропуски в столбце 'Cu_4F' средним значением: 0.48
Заполнены пропуски в столбце 'Ni_4F' средним значением: 2.84
Заполнены пропуски в столбце 'Ni_4.1C' средним значением: 4.69
Заполнены пропуски в столбце 'Ni_4.1C_max' средним значением: 4.98
Заполнены пропуски в столбце 'Ni_4.1C_min' средним значением: 4.03
Заполнены пропуски в столбце 'Ni_4.1T' средним значением: 0.73
Заполнены пропуски в столбце 'Ni_4.1T_max' средним значением: 0.78
Заполнены пропуски в столбце 'Ni_4.1T_min' средним значением: 0.63
Заполнены пропуски в столбце 'FM_4.1_A' средним значением: 0.68
Заполнены пропуски в столбце 'Ni_4.2C' средним значением: 4.20
Заполнены пропуски в столбце 'Ni_4.2C_max' средним значением: 4.59
Заполнены пропуски в столбце 'Ni_4.2C_min' средним значением: 3.69
Заполнены пропуски в столбце 'Ni_4.2T' средним значением: 0.76
Заполнены пропуски в столбце 'Ni

In [ ]:
# Проверка на пропущенные знаки в датасете
nikel_data.isnull().sum()

,0
MEAS_DT,0
Cu_oreth,0
Ni_oreth,0
Ore_mass,0
Mass_1,0
...,...
Ni_3.2C_max,0
Ni_3.2C_min,0
Cu_3.2T_max,0
Cu_3.2T_min,0


In [ ]:
# Проверяем количество строк до удаления
print(f"Количество строк до удаления: {len(nikel_data)}")

# Удаляем строки с пропущенными значениями
nikel_data_cleaned = nikel_data.dropna()

# Проверяем количество строк после удаления
print(f"Количество строк после удаления: {len(nikel_data_cleaned)}")

# Выводим очищенные данные
print("\nОчищенные данные:")
print(nikel_data_cleaned)

Количество строк до удаления: 30336
Количество строк после удаления: 30336

Очищенные данные:
       MEAS_DT  Cu_oreth  Ni_oreth  Ore_mass       Mass_1      Mass_2  \
0            0    2.6097    1.5313    1096.5  1240.597656  692.090942   
1            1    2.5548    1.4842    1123.0  1205.422363  693.616394   
2            2    2.5109    1.4355     840.0  1188.762573  698.350586   
3            3    2.4765    1.3852     824.0  1151.888672  714.678101   
4            4    2.3585    1.3368       0.0  1104.101318  730.190674   
...        ...       ...       ...       ...          ...         ...   
30331    30331    2.7133    1.4200    1231.0  1377.498413  766.228027   
30332    30332    2.7582    1.5234    1149.5  1412.641235  769.619324   
30333    30333    2.7487    1.5030    1210.5  1365.101562  785.184998   
30334    30334    2.7067    1.5257    1389.5  1382.615723  788.089050   
30335    30335    2.7335    1.5712    1370.0  1453.007324  786.615051   

         Dens_4      Mass_4  

In [ ]:
print(nikel_data_cleaned.columns)

Index(['MEAS_DT', 'Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Mass_1', 'Mass_2',
       'Dens_4', 'Mass_4', 'Vol_4', 'Cu_4F',
       ...
       'Cu_3.1T_max', 'Cu_3.1T_min', 'FM_3.2_A', 'Cu_3.2C_max', 'Cu_3.2C_min',
       'Ni_3.2C_max', 'Ni_3.2C_min', 'Cu_3.2T_max', 'Cu_3.2T_min', 'Ni_rec'],
      dtype='object', length=130)


In [ ]:
# Фильтруем необходимые признаки
# Обновляем список на основе доступных столбцов
selected_columns = [
    'Ni_rec', 'Cu_oreth', 'Ni_oreth', 'Ore_mass',
    'Mass_1', 'Mass_2', 'Dens_4', 'Mass_4',
    'Vol_4', 'Cu_4F'
]

# Создаем новый DataFrame только с нужными признаками
# Прежде чем это сделать, убедитесь, что все выбранные столбцы существуют
existing_columns = [col for col in selected_columns if col in nikel_data_cleaned.columns]

if not existing_columns:
    raise ValueError("Нет доступных признаков для фильтрации.")

filtered_data = nikel_data_cleaned[existing_columns]

# Генерируем 20 временных меток с 15-минутной дискретностью
time_stamps = pd.date_range(start='2024-01-01', periods=20, freq='15T')

# Создаем пустой DataFrame для результатов
test_df = pd.DataFrame(time_stamps, columns=['Time'])

# Генерация границ min и max для каждого признака
increments = {
    'Ni_rec': 0.1,
    'Cu_oreth': 0.1,
    'Ni_oreth': 0.1,
    'Ore_mass': 0.1,
    'Mass_1': 0.1,
    'Mass_2': 0.1,
    'Dens_4': 0.1,
    'Mass_4': 0.1,
    'Vol_4': 0.1,
    'Cu_4F': 0.1,
}

# Заполнение границ min и max для каждого из признаков
for col in existing_columns:
    increment = increments.get(col, 0.1)  # Используем значение по умолчанию, если нет в increments
    min_values = np.random.choice(np.arange(0, 1, increment), size=20)
    max_values = min_values + np.random.choice(np.arange(0.1, 1, increment), size=20)

    test_df[f'{col}_min'] = min_values
    test_df[f'{col}_max'] = max_values

<ipython-input-14-6ab7727c995a>:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_stamps = pd.date_range(start='2024-01-01', periods=20, freq='15T')


In [ ]:
# Сохраняем DataFrame в файл test_df.csv
test_df.to_csv('test_df.csv', index=False)

# Загружаем файл на ПК
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Подсчет коэффициента корреляции для всего датафрейма и округление значений
# получается корреляционная матрица
corr_matrix = nikel_data.corr().round(2)

# Сохранение корреляционной матрицы в Excel-файл
file_name = 'correlation_matrix.xlsx'
corr_matrix.to_excel(file_name, sheet_name='Correlation Matrix')

# Загрузка файла на ваш компьютер
files.download(file_name)

corr_matrix

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,MEAS_DT,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
MEAS_DT,1.00,-0.21,-0.16,-0.03,0.05,0.01,-0.09,-0.07,-0.00,0.02,...,0.82,-0.10,0.22,0.73,0.16,-0.87,NaN,0.90,-0.59,-0.19
Cu_oreth,-0.21,1.00,0.27,0.09,0.07,0.05,0.00,-0.00,-0.01,0.10,...,-0.25,-0.08,-0.15,-0.21,-0.10,0.23,NaN,-0.24,0.07,0.18
Ni_oreth,-0.16,0.27,1.00,0.16,0.15,0.11,0.08,0.09,0.00,0.02,...,-0.13,0.02,-0.01,0.01,0.04,0.08,NaN,-0.16,-0.05,0.32
Ore_mass,-0.03,0.09,0.16,1.00,0.79,0.66,0.60,0.62,0.64,-0.00,...,-0.04,-0.12,0.10,-0.05,0.07,-0.00,NaN,-0.05,0.03,0.04
Mass_1,0.05,0.07,0.15,0.79,1.00,0.86,0.75,0.78,0.80,-0.01,...,0.03,-0.19,0.17,-0.03,0.11,-0.07,NaN,0.02,-0.01,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ni_3.2C_max,-0.87,0.23,0.08,-0.00,-0.07,-0.05,0.03,0.01,0.02,-0.02,...,-0.84,0.06,-0.28,-0.71,-0.41,1.00,NaN,-0.82,0.49,0.17
Ni_3.2C_min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cu_3.2T_max,0.90,-0.24,-0.16,-0.05,0.02,-0.03,-0.10,-0.08,-0.03,-0.00,...,0.91,0.13,0.32,0.61,0.06,-0.82,NaN,1.00,-0.39,-0.19
Cu_3.2T_min,-0.59,0.07,-0.05,0.03,-0.01,-0.03,0.05,0.03,0.03,0.02,...,-0.41,0.52,-0.24,-0.67,-0.27,0.49,NaN,-0.39,1.00,0.02


### Анализ диапазона корреляции от -0.60 до +0.60

In [ ]:
# Выбираем интересующий нас столбец и все остальные столбцы, кроме Ni_rec
selected_cols = list(set(corr_matrix.columns) - {'Ni_rec'})

# Создаем подмножество корреляционной матрицы, где var1 = 'Ni_rec' и var2 - все остальные столбцы
subset_corrmatrix = corr_matrix[['Ni_rec']].join(corr_matrix[selected_cols])

# Преобразуем DataFrame в Series для удобного поиска
flat_corrmatrix = subset_corrmatrix.stack()

# Переносим данные в новый DataFrame и задаем имена колонок
filtered_flat_corrmatrix = flat_corrmatrix.reset_index()
filtered_flat_corrmatrix.columns = ['var1', 'var2', 'correlation']

# Убираем диагональные элементы (корреляция переменной самой с собой)
filtered_flat_corrmatrix = filtered_flat_corrmatrix.query("var1 == 'Ni_rec'")

# Сортируем по убыванию абсолютного значения корреляции
sorted_filtered_flat_corrmatrix = filtered_flat_corrmatrix.sort_values(by='correlation', key=lambda x: abs(x), ascending=False)

# Оставляем только те строки, где корреляция находится в диапазоне от -0.60 до 0.60
final_result = sorted_filtered_flat_corrmatrix.query('(-0.60 <= correlation) & (correlation <= 0.60)')

# Берем первые 20 строк (самые сильные связи)
top_N_correlations = final_result.head(20)

# Выводим результат
print(top_N_correlations)

         var1         var2  correlation
16625  Ni_rec     Ni_oreth         0.32
16543  Ni_rec  Ni_1.2T_max        -0.24
16554  Ni_rec      Ni_4.1C        -0.23
16525  Ni_rec  Ni_4.1C_max        -0.21
16632  Ni_rec  Cu_2.2T_max        -0.21
16576  Ni_rec  Ni_1.1C_min         0.21
16539  Ni_rec  Ni_1.1T_max        -0.20
16604  Ni_rec  Ni_6.2C_min        -0.20
16550  Ni_rec  Ni_5.2C_min        -0.20
16596  Ni_rec      Ni_6.1T        -0.19
16594  Ni_rec  Ni_6.2T_min        -0.19
16524  Ni_rec  Ni_4.1T_max        -0.19
16566  Ni_rec  Cu_3.2T_max        -0.19
16533  Ni_rec      MEAS_DT        -0.19
16581  Ni_rec  Ni_5.2C_max        -0.18
16603  Ni_rec  Ni_6.1C_min        -0.18
16613  Ni_rec  Cu_2.1T_max         0.18
16580  Ni_rec  Cu_2.1T_min         0.18
16633  Ni_rec     Cu_oreth         0.18
16568  Ni_rec  Ni_4.2C_max        -0.18


На основе предоставленных данных о корреляциях между различными признаками и значением `Ni_rec`, можно сделать несколько выводов:

### Основные выводы:

1. **Корреляционная связь**:
   - Наибольшая положительная корреляция наблюдается между `Ni_rec` и `Ni_oreth` (коэффициент корреляции 0.33). Это означает, что увеличение содержания никеля в руде положительно влияет на извлечение никеля в готовую продукцию.
   - Самая сильная отрицательная корреляция присутствует между `Ni_rec` и `Ni_1.2T_max` (-0.25). Это говорит о том, что максимальные значения хвостов первой флотомашины негативно влияют на извлечение никеля.
   
2. **Зависимость от других факторов**:
   - Значительные отрицательные корреляции наблюдаются также с такими признаками, как `Ni_4.1C`, `Ni_4.1C_max`, `Cu_2.2T_max`, `Ni_5.2C_min`, `Ni_1.1T_max`, `Ni_6.2C`, `Ni_6.2C_min`, `Ni_5F`, `MEAS_DT`, `Ni_5.1C`, `Ni_6.1T`, `Ni_6.1C_min`, `Ni_6.2T_min`, `Cu_3.2T_max`. Все эти факторы могут оказывать негативное влияние на извлечение никеля.
   - Положительное влияние оказывают такие признаки, как `Cu_2.1T`, `Cu_oreth`, `Cu_2.1T_min`.

3. **Практическое значение**:
   - Для улучшения процесса извлечения никеля рекомендуется обратить особое внимание на управление факторами, имеющими наибольшие положительные и отрицательные корреляции с `Ni_rec`. Например, контроль за содержанием никеля в руде (`Ni_oreth`) и минимизация потерь никеля в хвостах первой флотомашины (`Ni_1.2T_max`).
   - Также стоит учитывать влияние таких факторов, как концентрация меди в продуктах различных стадий флотации (`Cu_2.1T`, `Cu_2.2T_max`, `Cu_3.2T_max`), поскольку они тоже имеют значительную корреляцию с извлечением никеля.


### Отбор и выделение признаков

In [ ]:
# Отбор признаков на основе корреляционного анализа
features = ['Ni_oreth', 'Ni_1.2T_max', 'Ni_4.1C', 'Ni_1.1C_min', 'Ni_4.1C_max', 'Cu_2.1T', 'Cu_2.2T_max', 'Ni_5.2C_min', 'Ni_1.1T_max', 'Ni_6.2C', 'Ni_6.2C_min', 'Ni_5F', 'MEAS_DT', 'Ni_5.1C', 'Ni_6.1T', 'Ni_6.1C_min', 'Ni_6.2T_min', 'Cu_3.2T_max', 'Cu_2.1T_min', 'Cu_oreth']

# Выбор признаков
X = nikel_data[features].values

# Целевая переменная
y = nikel_data['Ni_rec'].values

In [ ]:
# разобьем данные на обучающую и тестовую выборку
# размер тестовой выборки составит 30%
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 42)

# посмотрим на новую размерность обучающей
print(X_train.shape, y_train.shape)

# и тестовой выборки
print(X_test.shape, y_test.shape)

(21235, 20) (21235,)
(9101, 20) (9101,)


In [ ]:
# Временная переменная для быстрой редакции вывода предсказанных/реальный значений в моделях
num_temp = 5

In [ ]:
print(np.unique(y_train))

[0.88470955 0.8866822  0.88785587 ... 0.983172   1.         1.        ]


### Деревья принятия решений при помощи GridSearchCV

In [ ]:
# Извлекаем уникальные значения из y_train
unique_values = np.unique(y_train)

# Проверяем, что y_train содержит дискретные метки классов
if len(unique_values) > 1 and np.issubdtype(y_train.dtype, np.integer):  # Проверяем на дискретные метки
    # Определение модели
    dt_model = DecisionTreeClassifier()

    # Определение сетки гиперпараметров
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [2, 5, 10]
    }

    # Создание экземпляра GridSearchCV
    grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

    # Поиск лучших гиперпараметров
    grid_result = grid_search.fit(X_train, y_train)

    # Вывод лучших результатов
    print("Accuracy rate of DecisionTreeClassifier: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
else:
    print("Целевая переменная y_train не содержит дискретных классов или имеет непрерывные значения.")

Целевая переменная y_train не содержит дискретных классов или имеет непрерывные значения.


In [ ]:
from sklearn.tree import DecisionTreeRegressor  # Импортируем DecisionTreeRegressor

# Определение модели
dt_model = DecisionTreeRegressor()  # Используем DecisionTreeRegressor для задач регрессии

# Определение сетки гиперпараметров
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],  # Подходящие критерии для регрессии
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10]
}

# Создание экземпляра GridSearchCV
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Поиск лучших гиперпараметров
grid_result = grid_search.fit(X_train, y_train)

# Вывод лучших результатов
print("Best Mean Squared Error: %f using %s" % (-grid_result.best_score_, grid_result.best_params_))